In [1]:
import torch
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


/home/utilisateur/Documents/dev_ia/fine-tune-classification/farid/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#exploration du dataset

df = pd.read_csv('/home/utilisateur/Documents/dev_ia/fine-tune-classification/dataset.csv')

df.head(1)


,Unnamed: 0,url,title,description,cat1,cat2,cat3
0,0,https://www.tourisme-cambresis.fr/1-les-templiers-coffre.html,"Aventure-jeu : ""Les Templiers du coffre d'or""","Le jeu aventure « Les templiers du coffre d’or » créé par la ville de Caudry en partenariat avec l’association Caudry Ma Passion débarque !\n\nConstruit autour d’un scénario original et vous entraînant dans un jeu de piste en autonomie dans le centre-ville de Caudry, ce jeu aventure vous permettra de passer 2h de divertissement en famille.\n\nVous pourrez vous procurer le sac contenant tout le nécessaire pour vous lancer dans cette aventure pour la somme de 15€ (un sac permet à une famille entière de jouer) auprès du Bureau d'Information Touristique de Caudry.",Jeu,Famille,Détente


In [3]:
#garder colonne description, cat1, cat2, cat3

df = df[['description', 'cat1', 'cat2', 'cat3']]

In [4]:
#supprimer les retours à la ligne dans la colonne description

df['description'] = df['description'].str.replace('\n', ' ')
df.head(1)


,description,cat1,cat2,cat3
0,"Le jeu aventure « Les templiers du coffre d’or » créé par la ville de Caudry en partenariat avec l’association Caudry Ma Passion débarque ! Construit autour d’un scénario original et vous entraînant dans un jeu de piste en autonomie dans le centre-ville de Caudry, ce jeu aventure vous permettra de passer 2h de divertissement en famille. Vous pourrez vous procurer le sac contenant tout le nécessaire pour vous lancer dans cette aventure pour la somme de 15€ (un sac permet à une famille entière de jouer) auprès du Bureau d'Information Touristique de Caudry.",Jeu,Famille,Détente


In [5]:



#supprimer colonne cat2 et cat3

drop_columns = ['cat2', 'cat3']
df = df.drop(drop_columns, axis=1)

#supprimer le manquant dans la colonne description

df = df.dropna(subset=['description'])

#Labeencoder cat1

label_encoder = LabelEncoder()

df['cat1_encoded'] = label_encoder.fit_transform(df['cat1'])




In [6]:
# initialize the tokenizer and model
# define the number of classes
num_labels = len(df['cat1_encoded'].unique())

tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")
model = AutoModelForSequenceClassification.from_pretrained("papluca/xlm-roberta-base-language-detection", num_labels=num_labels)

# tokenize the descriptions
inputs = tokenizer(df['description'].tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")

# convert the labels to tensors
labels = torch.tensor(df['cat1_encoded'].tolist())





In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
#entrainer le modèle

dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

#définir l'optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

#entrainer le modèle
def train(model, dataloader, optimizer, epochs=1):
    model.train()
    for epoch in range(epochs):
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, dataloader, optimizer)              



/home/utilisateur/Documents/dev_ia/fine-tune-classification/farid/venv/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
